# Initial Configs


In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# Imports

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

#import plotly for visualization
import chart_studio.plotly as py
import plotly.offline as pyoff
import plotly.graph_objs as go
import plotly.express as px
import sys
from IPython.core.display import display, HTML
sys.path.append('..')
pyoff.init_notebook_mode()

import matplotlib.cm as cm
from sklearn.metrics import silhouette_score, silhouette_samples

from tqdm import tqdm_notebook as tqdm

from bokeh.resources import INLINE
import bokeh.io
from bokeh import *

from src.utils.preprocessing import *

%load_ext autoreload
%autoreload 2

# Package configs

In [ ]:
pd.set_option('display.max_columns', None)
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('plotting.backend', 'pandas_bokeh')
bokeh.io.output_notebook(INLINE)

# Functions

In [ ]:
# Print rapido para dimensão do Dataframe
def SZ(df):
    print(f"""
--- Dimensão ---
Linhas:  {df.shape[0]}
Colunas: {df.shape[1]}""")

In [ ]:
# Print rapido para dTypes do Dataframe
def DT(df):
    print(f"""
--- DataTypes ---
{df.dtypes}
    """)

# Reading Files

In [ ]:
years = range(2016,2021)

df_datatran = pd.read_csv(f"../data/processed/df_datatran_2009_2020.csv", delimiter=",", encoding='iso-8859-1')

SZ(df_datatran)
DT(df_datatran)

In [ ]:
# df_pib = pd.read_excel('../data/raw/PIB dos Municípios - base de dados 2010-2017.xls')

In [ ]:
# df_populacao = pd.read_csv('../data/raw/populacao_por_municipio.csv')

In [ ]:
# df_depara_municipio = pd.read_excel('../data/raw/RELATORIO_DTB_BRASIL_MUNICIPIO.xls')

# Data Processing

## PIB Dataset - DEPRECATED

In [ ]:
# df_pib = df_pib[['Ano', 'Código do Município', 'Nome do Município', 'Código da Unidade da Federação', 'Sigla da Unidade da Federação', 'Nome da Unidade da Federação', 'Produto Interno Bruto, \na preços correntes\n(R$ 1.000)', 'Produto Interno Bruto per capita, \na preços correntes\n(R$ 1,00)']]
# 
# df_pib.columns = ['ano', 'id_municipio', 'municipio', 'id_uf', 'uf', 'nome_uf', 'pib', 'pib_per_capita']

In [ ]:
# df_pib = df_pib[df_pib['ano'] == 2017]

In [ ]:
# df_pib[['municipio', 'nome_uf']] = df_pib[['municipio', 'nome_uf']].apply(
#     lambda x: x.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.
#     decode('utf-8').str.replace('[^\w\s]', '').str.lower().str.strip())

Criando depara's

In [ ]:
# df_depara_uf = df_pib[['id_uf', 'uf', 'nome_uf']].drop_duplicates()

In [ ]:
# df_depara_municipio = df_pib[['id_municipio', 'municipio']].drop_duplicates()

In [ ]:
# df_depara_uf_municipio = df_pib[['id_municipio', 'municipio', 'id_uf', 'uf', 'nome_uf']].drop_duplicates()

População Dataset

In [ ]:
# df_populacao = df_populacao.merge(df_depara_municipio, how='left', on='id_municipio')

## Datatran Dataset

In [ ]:
df_datatran = make_dataset(df_datatran, years)

In [ ]:
# plot_data = [
#     go.Histogram(
#         x=df_datatran['risco']
#     )
# ]

# plot_layout = go.Layout(
#         title='Distribuição do risco'
#     )
# fig = go.Figure(data=plot_data, layout=plot_layout)
# pyoff.iplot(fig)

In [ ]:
# plot_data = [
#     go.Histogram(
#         x=np.log(df_datatran['risco'])
#     )
# ]
# 
# plot_layout = go.Layout(
#         title='Distribuição do risco'
#     )
# fig = go.Figure(data=plot_data, layout=plot_layout)
# pyoff.iplot(fig)

In [ ]:
# df_datatran['risco'] = df_datatran.groupby(['br', 'km'])['valor_1'].cumsum()
# df_acidentes_ate_data = df_datatran.groupby(['data_inversa', 'valor_1'])['valor_1'].sum()
# df_acidentes_ate_data = df_acidentes_ate_data.groupby(['valor_1']).cumsum().reset_index(name='qtd_acidentes_ate_data')
# df_datatran = df_datatran.merge(df_acidentes_ate_data[['data_inversa', 'qtd_acidentes_ate_data']], how='left', on='data_inversa')

Agrupar causas de acidentes para diminuir a cardinalidade do problema

In [ ]:
# dict_causa_acidente = {
#     'falta de atencao':'falta de atencao', 
#     'falta de atencao a conducao':'falta de atencao',
#     'dormindo':'dormindo', 
#     'condutor dormindo':'dormindo',
#     'deficiencia ou nao acionamento do sistema de iluminacaosinalizacao do veiculo': 'defeito mecanico', 
#     'defeito mecanico em veiculo':'defeito mecanico', 
#     'avarias eou desgaste excessivo no pneu':'defeito mecanico', 
#     'defeito mecanico no veiculo':'defeito mecanico',
#     'ingestao de alcool':'ingestao de substancias psicoativas', 
#     'ingestao de substancias psicoativas':'ingestao de substancias psicoativas',
#     'restricao de visibilidade':'restricao de visibilidade', 
#     'sinalizacao da via insuficiente ou inadequada':'restricao de visibilidade',
#     'outras':'outras', 
#     'animais na pista':'animais na pista', 
#     'desobediencia a sinalizacao':'desobediencia a sinalizacao', 
#     'velocidade incompativel':'velocidade incompativel', 
#     'ultrapassagem indevida':'ultrapassagem indevida', 
#     'nao guardar distancia de seguranca':'nao guardar distancia de seguranca', 
#     'defeito na via':'defeito na via', 
#     'fenomenos da natureza':'fenomenos da natureza', 
#     'desobediencia as normas de transito pelo condutor':'desobediencia as normas de transito pelo condutor', 
#     'pista escorregadia':'pista escorregadia', 
#     'mal subito':'mal subito', 
#     'carga excessiva eou mal acondicionada':'carga excessiva eou mal acondicionada', 
#     'objeto estatico sobre o leito carrocavel':'objeto estatico sobre o leito carrocavel', 
#     'agressao externa':'agressao externa', 
#     'falta de atencao do pedestre':'falta de atencao do pedestre', 
#     'desobediencia as normas de transito pelo pedestre':'desobediencia as normas de transito pelo pedestre', 
#     'ingestao de alcool eou substancias psicoativas pelo pedestre':'ingestao de alcool eou substancias psicoativas pelo pedestre'
# }

In [ ]:
# df_datatran['causa_acidente'] = df_datatran.apply(lambda x: dict_causa_acidente[x['causa_acidente']], axis=1)

Agrupar tipo de acidentes para diminuir a cardinalidade do problema

In [ ]:
# dict_tipo_acidente = {
#     'colisao traseira': 'colisao',
#     'saida de pista': 'capotamento',
#     'colisao com objeto fixo': 'colisao',
#     'colisao transversal': 'colisao',
#     'tombamento': 'capotamento',
#     'queda de motocicleta  bicicleta  veiculo': 'capotamento',
#     'atropelamento de animal': 'atropelamento', 
#     'capotamento': 'capotamento',
#     'colisao lateral': 'colisao',
#     'colisao frontal': 'colisao',
#     'atropelamento de pessoa': 'atropelamento',
#     'colisao com objeto movel': 'colisao',
#     'danos eventuais': 'outras',
#     'incendio': 'outras',
#     'colisao com bicicleta': 'colisao',
#     'derramamento de carga': 'outras',
#     'queda de ocupante de veiculo': 'atropelamento',
#     'colisao com objeto estatico': 'colisao',
#     'saida de leito carrocavel': 'outras',
#     'atropelamento de pedestre': 'atropelamento',
#     'colisao com objeto em movimento': 'colisao',
#     'engavetamento': 'colisao'
# }

In [ ]:
# df_datatran['tipo_acidente'] = df_datatran.apply(lambda x: dict_tipo_acidente[x['tipo_acidente']], axis=1)

Feriados

In [ ]:
# df_datatran['dia'] = df_datatran['data_inversa'].dt.day
# df_datatran['mes'] = df_datatran['data_inversa'].dt.month
# df_datatran['ano'] = df_datatran['data_inversa'].dt.year
# df_datatran['diadoano'] = df_datatran['data_inversa'].dt.dayofyear

In [ ]:
# plot_data = [
#     go.Histogram(
#         x=df_datatran['diff_ac_feriado_dias']
#     )
# ]
# 
# plot_layout = go.Layout(
#         title='Número acidentes por janela feriado'
#     )
# fig = go.Figure(data=plot_data, layout=plot_layout)
# pyoff.iplot(fig)

In [ ]:
# plot_data = [
#     go.Histogram(
#         x=df_datatran['holiday_nome']
#     )
# ]
# 
# plot_layout = go.Layout(
#         title='Número acidentes por feriado'
#     )
# fig = go.Figure(data=plot_data, layout=plot_layout)
# pyoff.iplot(fig)

# Clustering

## K-Means

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# 
# lb_make = LabelEncoder()
# tx_data["SEGMENTO_AZURE_CODE"] = lb_make.fit_transform(tx_data["SEGMENTO_AZURE"])

In [ ]:
df_datatran = df_datatran.loc[df_datatran['data_inversa'] < '2020-01-01', :]

In [ ]:
def nb_cluster_silhouette_method(n_clusters):
    # Create a subplot with 1 row and 2 columns
    fig, (ax1, ax2) = plt.subplots(1, 2)
    fig.set_size_inches(18, 7)

    # The 1st subplot is the silhouette plot
    ax1.set_xlim([-1, 1])
    # The (n_clusters+1)*10 is for inserting blank space between silhouette plots of individual clusters, to demarcate them clearly.
    ax1.set_ylim([0, df_kmeans.shape[0] + (n_clusters + 1) * 10])

    # Initialize the clusterer with n_clusters value and a random generator seed of 42 for reproducibility.
    clusterer = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = clusterer.fit_predict(df_kmeans)

    # The silhouette_score gives the average value for all the samples.
    # This gives a perspective into the density and separation of the formed clusters
    silhouette_avg = silhouette_score(df_kmeans, cluster_labels)
    print("For n_clusters =", n_clusters,
          "The average silhouette_score is :", silhouette_avg)

    # Compute the silhouette scores for each sample
    sample_silhouette_values = silhouette_samples(df_kmeans, cluster_labels)

    y_lower = 10
    for i in range(n_clusters):
        # Aggregate the silhouette scores for samples belonging to cluster i, and sort them
        ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]

        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper),
                          0, ith_cluster_silhouette_values,
                          facecolor=color, edgecolor=color, alpha=0.7)

        # Label the silhouette plots with their cluster numbers at the middle
        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

        # Compute the new y_lower for next plot
        y_lower = y_upper + 10  # 10 for the 0 samples

    ax1.set_title("The silhouette plot for the various clusters.")
    ax1.set_xlabel("The silhouette coefficient values")
    ax1.set_ylabel("Cluster label")

    # The vertical line for average silhouette score of all the values
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-1, -0.8, -0.6, -0.4, -0.2, 0, 0.2, 0.4, 0.6, 0.8, 1])

    # 2nd Plot showing the actual clusters formed
    colors = cm.nipy_spectral(cluster_labels.astype(float) / n_clusters)
    ax2.scatter(df_kmeans['latitude'], df_kmeans['longitude'], marker='.', s=30, lw=0, alpha=0.7,
                c=colors, edgecolor='k')

    # Labeling the clusters
    centers = clusterer.cluster_centers_
    # Draw white circles at cluster centers
    ax2.scatter(centers[:, 0], centers[:, 1], marker='o',
                c="white", alpha=1, s=200, edgecolor='k')

    for i, c in enumerate(centers):
        ax2.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,
                    s=50, edgecolor='k')

    ax2.set_title("The visualization of the clustered data.")
    ax2.set_xlabel("Feature space for the 1st feature")
    ax2.set_ylabel("Feature space for the 2nd feature")

    plt.suptitle(("Silhouette analysis for KMeans clustering on sample data "
                  "with n_clusters = %d" % n_clusters),
                 fontsize=14, fontweight='bold')

plt.show()

In [ ]:
# for i in range(2,11):
#     nb_cluster_silhouette_method(i)

In [ ]:
df_kmeans['cluster_coords'].value_counts().plot(kind='bar', title='Qtde amostras por cluster')

In [ ]:
df_datatran = df_datatran.merge(df_kmeans[['latitude', 'longitude', 'risco_morte', 'cluster_coords']], how='left', on=['latitude','longitude','risco_morte'])

In [ ]:
df_datatran.drop_duplicates(subset='id', inplace=True)

In [ ]:
fig = px.scatter(df_datatran, x="longitude", y="latitude", color="cluster_coords", title='Cluster de coordenadas', width=500, height=500)
fig.show()

## DBSCAN

In [ ]:
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint

In [ ]:
kms_per_radian = 6371.0088
epsilon = 1.5 / kms_per_radian
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine').fit(np.radians(df_kmeans))
cluster_labels = db.labels_
num_clusters = len(set(cluster_labels))
clusters = pd.Series([df_kmeans[cluster_labels == n] for n in range(num_clusters)])
print('Number of clusters: {}'.format(num_clusters))

In [ ]:
# Compute DBSCAN
db = DBSCAN(eps=0.3, min_samples=10).fit(df_kmeans)
core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
core_samples_mask[db.core_sample_indices_] = True
labels = db.labels_

# Number of clusters in labels, ignoring noise if present.
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_noise_ = list(labels).count(-1)

print('Estimated number of clusters: %d' % n_clusters_)
print('Estimated number of noise points: %d' % n_noise_)

In [ ]:
# Black removed and is used for noise instead.
unique_labels = set(labels)
colors = [plt.cm.Spectral(each)
          for each in np.linspace(0, 1, len(unique_labels))]
for k, col in zip(unique_labels, colors):
    if k == -1:
        # Black used for noise.
        col = [0, 0, 0, 1]

    class_member_mask = (labels == k)

    xy = df_kmeans[class_member_mask & core_samples_mask]
    plt.plot(xy['latitude'], xy['longitude'], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=14)

    xy = df_kmeans[class_member_mask & ~core_samples_mask]
    plt.plot(xy['latitude'], xy['longitude'], 'o', markerfacecolor=tuple(col),
             markeredgecolor='k', markersize=6)

plt.title('Estimated number of clusters: %d' % n_clusters_)
plt.show()

In [ ]:
df_kmeans[class_member_mask & ~core_samples_mask]

Drop unwanted columns

In [ ]:
drop_columns = [
    'horario'
    , 'br'
    , 'km'
    , 'municipio'
    , 'causa_acidente'
    , 'tipo_acidente'
    , 'classificacao_acidente'
    , 'ano'
    , 'mortos'
    , 'feridos_leves'
    , 'feridos_graves'
    , 'ilesos'
    , 'ignorados'
    , 'feridos'
    , 'veiculos'
    , 'latitude'
    , 'longitude'
    , 'regional'
    , 'delegacia'
    , 'uop'
    , 'd'
    , 'ufbr'
]

df_datatran.drop(drop_columns, axis=1, inplace=True)
df_datatran.head()

In [ ]:
df_datatran.isnull().sum().sum()

In [ ]:
df_datatran_2020 = df_datatran.loc[df_datatran['data_inversa'] >= '2020-01-01', :]

# Under/Oversampling 

In [ ]:
#df_SMOTE.to_parquet('../data/processed/datasetSMOTENC2020.parquet', index=False)

In [ ]:
#df_class_0_under = df_class_0.sample(count_class_1)
#df_datatran_under = pd.concat([df_class_0_under, df_class_1], axis=0)

#print('Random under-sampling:')
#print(df_datatran_under['contem_vitima_fatal'].value_counts())

#df_datatran_under['contem_vitima_fatal'].value_counts().plot(kind='bar', title='Count (target)');

Oversampling por cópia simples - deprecated

In [ ]:
#df_SMOTE['contem_vitima_fatal'].value_counts().plot(kind='bar', title='Count (target)');

Oversampling SMOTENC

In [ ]:
cat_columns = [
      'dia_semana'
    , 'fase_dia'
    , 'sentido_via'
    , 'condicao_metereologica'
    , 'tipo_pista'
    , 'tracado_via'
    , 'uf'
    , 'uso_solo'
    , 'em_janela_feriado'
]

In [ ]:
all_clusters = df_datatran['cluster_coords'].unique()
all_clusters.sort()
all_clusters

In [ ]:
from imblearn.over_sampling import SMOTENC
from sklearn.model_selection import train_test_split

#Splitando os dados de teste e treino para balancear apenas os dados de treino
new_df2_split = df_datatran.drop(['data_inversa'], axis=1)

# list_df_train_resampled = []
# list_df_test = []

# for cluster in all_clusters:
#     df_cluster = new_df2_split.loc[new_df2_split['cluster_coords']==cluster, :]

#     new_df_split_train, df_split_test, target_df_split_train, target_df_split_test = train_test_split(df_cluster.drop(['contem_vitima_fatal'], axis=1), df_cluster['contem_vitima_fatal'], test_size=0.3, stratify=df_cluster[['contem_vitima_fatal', 'cluster_coords']])

#     df_split_test['contem_vitima_fatal'] = target_df_split_test

#     cat_col_indices = []
#     for cat in cat_columns:
#         cat_col_indices.append(new_df_split_train.columns.get_loc(cat))
#     cat_col_indices = sorted(cat_col_indices)
    
#     smote_nc = SMOTENC(categorical_features = cat_col_indices)#,k_neighbors = 100)

#     df_train_resampled, target_df_resampled = smote_nc.fit_resample(new_df_split_train, target_df_split_train)
#     df_train_resampled['contem_vitima_fatal'] = target_df_resampled

#     list_df_train_resampled.append(df_train_resampled)
#     list_df_test.append(df_split_test)

# df_train_resampled = pd.concat(list_df_train_resampled)
# df_test = pd.concat(list_df_test)

new_df_split_train, df_split_test, target_df_split_train, target_df_split_test = train_test_split(new_df2_split.drop(['contem_vitima_fatal'], axis=1), new_df2_split['contem_vitima_fatal'], test_size=0.3, stratify=new_df2_split[['contem_vitima_fatal', 'cluster_coords']])

df_split_test['contem_vitima_fatal'] = target_df_split_test

cat_col_indices = []
for cat in cat_columns:
    cat_col_indices.append(new_df_split_train.columns.get_loc(cat))
cat_col_indices = sorted(cat_col_indices)

smote_nc = SMOTENC(categorical_features = cat_col_indices)#,k_neighbors = 100)

df_train_resampled, target_df_resampled = smote_nc.fit_resample(new_df_split_train, target_df_split_train)
df_train_resampled['contem_vitima_fatal'] = target_df_resampled

df_test = df_split_test

In [ ]:
df_datatran['contem_vitima_fatal'].value_counts().plot(kind='bar', title='Antes Balanceamento');

In [ ]:
df_train_resampled['contem_vitima_fatal'].value_counts().plot(kind='bar', title='Depois Balancemento');

# Export de dados para modelagem

In [ ]:
SZ(df_datatran)

In [ ]:
SZ(df_train_resampled)

In [ ]:
SZ(df_test)

In [ ]:
df_datatran.to_parquet('../data/processed/datasetCompDesb_v1.parquet', index=False)
df_train_resampled.to_parquet('../data/processed/dataset_SMOTENC_TREINO_v1.parquet', index=False)
df_test.to_parquet('../data/processed/dataset_SMOTENC_TESTE_v1.parquet', index=False)

## FIM DA GERAÇÃO DE DADOS PARA MODELAGEM

## Teste dataset desbalanceado para SMOTE

In [ ]:
df_datatran_2020.to_parquet('../data/processed/datasetSMOTENC2020_v1.parquet', index=False)

In [ ]:
df_datatran_2020['contem_vitima_fatal'].value_counts().plot(kind='bar', title='Count (target)');

## Join Datasets - PIB DEPRECATED

In [ ]:
#df_final = df_datatran.merge(df_pib[['municipio', 'uf', 'id_municipio', 'pib', 'pib_per_capita']], how='left', on=['uf', 'municipio'])

In [ ]:
#df_final = df_final.merge(df_populacao, how='left', on=['ano', 'id_municipio', 'municipio'])

In [ ]:
#df_final[['pib', 'pib_per_capita', 'populacao']] = df_final[['uf', 'pib', 'pib_per_capita', 'populacao']].groupby("uf").transform(lambda x: x.fillna(x.mean()))

In [ ]:
#df_final.head()

In [ ]:
# df_final.to_csv('../data/processed/dataset_v4.csv', index=False, encoding='iso-8859-1')
# df_final.to_parquet('../data/processed/dataset_v4.parquet', index=False)

# Exploratory Analysis - DEPRECATED

In [ ]:
#df_datatran[['pessoas', 'mortos', 'feridos_leves', 'feridos_graves', 'ilesos', 'ignorados', 'feridos', 'veiculos']].describe()

In [ ]:
#df_datatran.head()

In [ ]:
#df_datatran_causa_acidente_count = df_datatran.groupby(['causa_acidente'])['id'].count().reset_index()

#df_datatran_causa_acidente_count.set_index('causa_acidente', inplace=True)

In [ ]:
#df_datatran_causa_acidente_count.sort_values(by='id', ascending=True).plot(kind='barh', figsize=(10,10))